<a href="https://www.kaggle.com/code/sinemkarahan/suggestion-system?scriptVersionId=224276913" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-product-reviews/Reviews.csv


In [2]:
import os

# Veri setinin bulunduğu dizini listeler
# Bu dizin, Kaggle Notebook'ta veri setlerinin varsayılan olarak bulunduğu dizindir
base_path = '/kaggle/input/'

# Mevcut veri seti klasörlerini ve dosyalarını listeler
for dirname, _, filenames in os.walk(base_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/amazon-product-reviews/Reviews.csv


In [3]:
import pandas as pd

# Veri setini doğru yoldan yükleyelim
data = pd.read_csv('/kaggle/input/amazon-product-reviews/Reviews.csv')

# İlk birkaç satırı inceleyelim
data.head()


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
# Eksik değerleri kontrol etme
missing_values = data.isnull().sum()
print(missing_values)

# Eksik değer içeren satırları kaldırma
data = data.dropna()


Id                         0
ProductId                  0
UserId                     0
ProfileName               26
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64


In [5]:
import pandas as pd
import re

# Temel bir stopwords listesi oluşturalım
custom_stopwords = [
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
    'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
    'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
    'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
    'was', 'were', 'be', 'been', 'being', 'if', 'then', 'that', 'the', 'and', 'but', 'or',
    'as', 'at', 'for', 'by', 'with', 'about', 'against', 'between', 'into', 'through',
    'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 
    'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
    'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
    'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
    'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now'
]

# Eksik değerleri dolduralım
data['ProfileName'] = data['ProfileName'].fillna('Bilinmiyor')
data['Summary'] = data['Summary'].fillna('Bilinmiyor')
data['Text'] = data['Text'].fillna('Bilinmiyor')

def clean_text(text):
    # Küçük harfe çevir
    text = text.lower()
    # Noktalama işaretlerini ve rakamları temizle
    text = re.sub(r'[^a-z\s]', '', text)
    # Stopwords'leri temizle
    text = ' '.join(word for word in text.split() if word not in custom_stopwords)
    return text

# 'Summary' ve 'Text' sütunlarını temizleyelim
data['Cleaned_Summary'] = data['Summary'].apply(clean_text)
data['Cleaned_Text'] = data['Text'].apply(clean_text)

# Temizlenmiş verileri kontrol edelim
cleaned_data_preview = data[['Cleaned_Summary', 'Cleaned_Text']].head()
print(cleaned_data_preview)







         Cleaned_Summary                                       Cleaned_Text
0  good quality dog food  have bought several of vitality canned dog foo...
1             advertised  product arrived labeled jumbo salted peanutsth...
2           delight says  a confection has around a centuries a light pi...
3         cough medicine  looking secret ingredient robitussin believe h...
4            great taffy  great taffy a great price a wide assortment of...


In [6]:
from sklearn.model_selection import train_test_split

# Özellikler ve hedef değişken
X = data['Cleaned_Text']
y = data['Score']  # ya da uygun bir hedef değişkeni

# Eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TfidfVectorizer ile metin verisini sayısal hale getirme
vectorizer = TfidfVectorizer(max_features=5000)  # En fazla 5000 kelime ile sınırlıyoruz
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)  # Test verisini dönüştür



In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Naive Bayes modelini oluşturma ve eğitme
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)

# Test seti ile tahmin yapma
y_pred = nb_model.predict(X_test_tfidf)

# Model performansını değerlendirme
print("Doğruluk Skoru:", accuracy_score(y_test, y_pred))
print("\nSınıflandırma Raporu:\n", classification_report(y_test, y_pred))


Doğruluk Skoru: 0.6717745269658079

Sınıflandırma Raporu:
               precision    recall  f1-score   support

           1       0.71      0.32      0.44     10515
           2       0.58      0.01      0.02      5937
           3       0.51      0.02      0.04      8460
           4       0.45      0.03      0.06     16026
           5       0.67      0.99      0.80     72743

    accuracy                           0.67    113681
   macro avg       0.58      0.27      0.27    113681
weighted avg       0.63      0.67      0.57    113681



In [9]:
# Test yorumları
sample_reviews = [
    "This product is amazing, I loved it!",  # Olumlu bir yorum
    "The quality is really bad, I'm disappointed.",  # Olumsuz bir yorum
    "It's okay, not the best but works.",  # Nötr bir yorum
    "Terrible experience, would not recommend."  # Çok olumsuz bir yorum
]

# Yorumları sayısal hale getirme
sample_reviews_tfidf = vectorizer.transform(sample_reviews)

# Model ile tahmin yapma
predictions = nb_model.predict(sample_reviews_tfidf)

# Sonuçları yazdırma
for review, pred in zip(sample_reviews, predictions):
    print(f"Review: '{review}' -> Tahmin edilen sınıf: {pred}")


Review: 'This product is amazing, I loved it!' -> Tahmin edilen sınıf: 5
Review: 'The quality is really bad, I'm disappointed.' -> Tahmin edilen sınıf: 5
Review: 'It's okay, not the best but works.' -> Tahmin edilen sınıf: 5
Review: 'Terrible experience, would not recommend.' -> Tahmin edilen sınıf: 1


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Kelime frekanslarını hesaplamak için CountVectorizer kullan
count_vectorizer = CountVectorizer(stop_words='english')
X_counts = count_vectorizer.fit_transform(data['Cleaned_Text'])
sum_words = X_counts.sum(axis=0)  # Kelime frekanslarını topla
words_freq = [(word, sum_words[0, idx]) for word, idx in count_vectorizer.vocabulary_.items()]
words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)

# En sık geçen kelimeleri göster
df_words = pd.DataFrame(words_freq, columns=['Word', 'Frequency'])
print(df_words.head(10))


      Word  Frequency
0       br     272674
1     like     251867
2     good     195341
3    taste     166576
4    great     163565
5   coffee     160186
6  product     146450
7   flavor     142559
8      tea     133105
9     love     126644


In [11]:
from textblob import TextBlob

# Duygu analizini gerçekleştiren fonksiyon
def analyze_sentiment(review):
    analysis = TextBlob(review)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity < 0:
        return 'negative'
    else:
        return 'neutral'


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Yorumlardan anahtar kelime çıkarma
def extract_keywords(reviews):
    vectorizer = TfidfVectorizer(max_features=10)  # En fazla 10 anahtar kelime çıkar
    X = vectorizer.fit_transform(reviews)
    return vectorizer.get_feature_names_out()

# Örnek kullanımı
reviews = ["The product has great taste but is too expensive.",
           "The quality of the material is poor."]
keywords = extract_keywords(reviews)
print(keywords)


['but' 'expensive' 'great' 'has' 'is' 'material' 'of' 'poor' 'product'
 'the']


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk

# Eğer ilk kez kullanıyorsanız aşağıdaki satırı çalıştırın:
# nltk.download('stopwords')

# Stopwords listesini alıyoruz
stop_words = list(stopwords.words('english'))

# TF-IDF vektörleştiriciyi stopwords ile güncelliyoruz
vectorizer = TfidfVectorizer(max_features=10, stop_words=stop_words)
X = vectorizer.fit_transform(data['Cleaned_Text'])

# Anahtar kelimeleri yeniden çıkarma
keywords = vectorizer.get_feature_names_out()
print(keywords)



['br' 'coffee' 'flavor' 'good' 'great' 'like' 'one' 'product' 'taste'
 'tea']


In [14]:
from textblob import TextBlob

# Yorumun pozitif, negatif veya nötr olup olmadığını belirleyen fonksiyon
def analyze_sentiment(review):
    analysis = TextBlob(review)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity < 0:
        return 'negative'
    else:
        return 'neutral'

# Yorumlar üzerinde uygulama
data['Sentiment'] = data['Cleaned_Text'].apply(analyze_sentiment)


In [15]:
# Sonuçları kontrol et
print(data[['Cleaned_Text', 'Sentiment']].head(10))


                                        Cleaned_Text Sentiment
0  have bought several of vitality canned dog foo...  positive
1  product arrived labeled jumbo salted peanutsth...  positive
2  a confection has around a centuries a light pi...  positive
3  looking secret ingredient robitussin believe h...  positive
4  great taffy a great price a wide assortment of...  positive
5  got a wild hair taffy ordered five pound bag t...  positive
6  saltwater taffy had great flavors soft chewy c...  positive
7  taffy good soft chewy flavors amazing would de...  positive
8  right im mostly sprouting cats eat grass love ...  positive
9  a healthy dog food good digestion also good sm...  positive


In [16]:
# Örnek bir yorum için
sample_review = "This product is bad!"
print(analyze_sentiment(sample_review))  # 'positive' bekleniyor


negative


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Yorumları alıyoruz (Cleaned_Text sütunu)
corpus = data['Cleaned_Text']

# TF-IDF vektörleştirici ile anahtar kelimeleri çıkarma
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=0.05, stop_words='english', ngram_range=(1,2))
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# Anahtar kelimeleri alıyoruz
feature_names = tfidf_vectorizer.get_feature_names_out()

# Anahtar kelimeleri DataFrame'e çeviriyoruz
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# Yorum başına en yüksek TF-IDF skora sahip kelimeleri alıyoruz
important_keywords = tfidf_df.idxmax(axis=1)
data['Keywords'] = important_keywords

print(data[['Cleaned_Text', 'Sentiment', 'Keywords']].head())


                                        Cleaned_Text Sentiment Keywords
0  have bought several of vitality canned dog foo...  positive   better
1  product arrived labeled jumbo salted peanutsth...  positive  product
2  a confection has around a centuries a light pi...  positive    sugar
3  looking secret ingredient robitussin believe h...  positive  ordered
4  great taffy a great price a wide assortment of...  positive    great


In [18]:
import pandas as pd

data = pd.read_csv('/kaggle/input/amazon-product-reviews/Reviews.csv')


In [19]:
print(data.columns)


Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')


In [20]:
data['Sentiment'] = data['Text'].apply(analyze_sentiment)


In [21]:
print(analyze_sentiment("This product is great!"))


positive


In [22]:
data['Sentiment'] = data['Text'].apply(analyze_sentiment)


In [23]:
print(data[['Text', 'Sentiment']].head())


                                                Text Sentiment
0  I have bought several of the Vitality canned d...  positive
1  Product arrived labeled as Jumbo Salted Peanut...  negative
2  This is a confection that has been around a fe...  positive
3  If you are looking for the secret ingredient i...  positive
4  Great taffy at a great price.  There was a wid...  positive


In [24]:
import pandas as pd

# DataFrame'iniz burada, örneğin:
data = pd.read_csv('/kaggle/input/amazon-product-reviews/Reviews.csv')

# Duygu analizi uygulama fonksiyonu (örnek)
def analyze_sentiment(text):
    # Basit bir duygu analizi
    if "not" in text or "bad" in text:
        return 'negative'
    elif "good" in text or "great" in text:
        return 'positive'
    else:
        return 'neutral'

# Duygu analizini uygulama
data['Sentiment'] = data['Text'].apply(analyze_sentiment)

# Öneri oluşturma fonksiyonu
def generate_suggestion(sentiment, review_text):
    if sentiment == 'negative':
        return f"Improvement could be made regarding the issues mentioned in your review: '{review_text}'."
    elif sentiment == 'neutral':
        return f"Consider enhancing the aspects mentioned in your review: '{review_text}'."
    else:
        return f"Thank you for your positive feedback on: '{review_text}'!"

# Her yorum için öneri oluşturma
data['Suggestion'] = data.apply(lambda row: generate_suggestion(row['Sentiment'], row['Text']), axis=1)

# Sonuçları görüntüleme
print(data[['Text', 'Sentiment', 'Suggestion']].head())


                                                Text Sentiment  \
0  I have bought several of the Vitality canned d...  positive   
1  Product arrived labeled as Jumbo Salted Peanut...   neutral   
2  This is a confection that has been around a fe...   neutral   
3  If you are looking for the secret ingredient i...  positive   
4  Great taffy at a great price.  There was a wid...  positive   

                                          Suggestion  
0  Thank you for your positive feedback on: 'I ha...  
1  Consider enhancing the aspects mentioned in yo...  
2  Consider enhancing the aspects mentioned in yo...  
3  Thank you for your positive feedback on: 'If y...  
4  Thank you for your positive feedback on: 'Grea...  


In [25]:
# Güncellenmiş öneri oluşturma fonksiyonu
def generate_detailed_suggestion(sentiment, review_text):
    if sentiment == 'negative':
        return (f"We appreciate your feedback. To improve your experience, could you please share more details about the issues you faced in: '{review_text}'?")
    elif sentiment == 'neutral':
        return (f"Thank you for your insights. We encourage you to explore additional features or products that might meet your expectations regarding: '{review_text}'.")
    else:
        return (f"Thank you for your positive feedback! We're thrilled to hear you enjoyed: '{review_text}'. You might also like to try our other flavors or products!")

# Her yorum için öneri oluşturma
data['Suggestion'] = data.apply(lambda row: generate_detailed_suggestion(row['Sentiment'], row['Text']), axis=1)

# Sonuçları görüntüleme
print(data[['Text', 'Sentiment', 'Suggestion']].head())


                                                Text Sentiment  \
0  I have bought several of the Vitality canned d...  positive   
1  Product arrived labeled as Jumbo Salted Peanut...   neutral   
2  This is a confection that has been around a fe...   neutral   
3  If you are looking for the secret ingredient i...  positive   
4  Great taffy at a great price.  There was a wid...  positive   

                                          Suggestion  
0  Thank you for your positive feedback! We're th...  
1  Thank you for your insights. We encourage you ...  
2  Thank you for your insights. We encourage you ...  
3  Thank you for your positive feedback! We're th...  
4  Thank you for your positive feedback! We're th...  


In [26]:
import pandas as pd
from textblob import TextBlob

# Veri setini yükleme
data = pd.read_csv('/kaggle/input/amazon-product-reviews/Reviews.csv')

# Duygu analizi fonksiyonu
def analyze_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity < 0:
        return 'negative'
    else:
        return 'neutral'

# Duygu analizi uygulama
data['Sentiment'] = data['Text'].apply(analyze_sentiment)

# Öneri oluşturma fonksiyonu
def generate_detailed_suggestion(sentiment, review_text):
    if sentiment == 'negative':
        return (f"We appreciate your feedback on: '{review_text}'. We're sorry to hear about your experience. Can you please share more details so we can address this issue?")
    elif sentiment == 'neutral':
        return (f"Thank you for your insights on: '{review_text}'. To enhance your experience, we recommend exploring additional features or products.")
    else:  # positive
        return (f"Thank you for your wonderful feedback on: '{review_text}'. We're thrilled you enjoyed it! Have you tried our other products?")

# Öneri sütununu doldurma
data['Suggestion'] = data.apply(lambda row: generate_detailed_suggestion(row['Sentiment'], row['Text']), axis=1)

# Çıktıyı görüntüleme
print(data[['Text', 'Sentiment', 'Suggestion']].head())


                                                Text Sentiment  \
0  I have bought several of the Vitality canned d...  positive   
1  Product arrived labeled as Jumbo Salted Peanut...  negative   
2  This is a confection that has been around a fe...  positive   
3  If you are looking for the secret ingredient i...  positive   
4  Great taffy at a great price.  There was a wid...  positive   

                                          Suggestion  
0  Thank you for your wonderful feedback on: 'I h...  
1  We appreciate your feedback on: 'Product arriv...  
2  Thank you for your wonderful feedback on: 'Thi...  
3  Thank you for your wonderful feedback on: 'If ...  
4  Thank you for your wonderful feedback on: 'Gre...  


In [27]:
import random

# Kullanıcı isimleri ve alışveriş geçmişini simüle etme
user_names = ['Alice', 'Bob', 'Charlie', 'Daisy', 'Eve']
purchase_histories = ["Vitality canned dog food", "Jumbo Salted Peanuts", "Confection", "Secret ingredient", "Taffy"]

# Rastgele isim ve geçmiş ekleyerek veri çerçevesini doldurma
data['User_Name'] = [random.choice(user_names) for _ in range(len(data))]
data['Purchase_History'] = [random.choice(purchase_histories) for _ in range(len(data))]

# Geliştirilmiş öneri fonksiyonu
def generate_personalized_suggestion(sentiment, review_text, user_name, purchase_history):
    keywords = ["delivery", "quality", "taste", "packaging", "price"]
    matched_keywords = [word for word in keywords if word in review_text.lower()]

    if sentiment == 'negative':
        if matched_keywords:
            suggestion = (f"{user_name}, we appreciate your feedback on your purchase of '{purchase_history}'. "
                          f"We're sorry to hear about issues with {', '.join(matched_keywords)}. "
                          "Our team is working to address these concerns. Meanwhile, please check if there's an alternative product that may suit your needs.")
        else:
            suggestion = (f"{user_name}, we're sorry to hear about your experience with '{purchase_history}'. "
                          "We appreciate more details to help us improve.")
        suggestion += " Please consider reaching out to our support team for immediate assistance."

    elif sentiment == 'neutral':
        suggestion = (f"Thank you for your insights on '{purchase_history}', {user_name}. "
                      "We recommend exploring related products or additional features to enhance your experience.")

    else:  # positive
        suggestion = (f"Thank you for your wonderful feedback, {user_name}! We're thrilled that you enjoyed '{purchase_history}'. "
                      "Have you tried similar products in our range? We think you might enjoy them too!")

    return suggestion

# Güncellenmiş öneri sütununu doldurma
data['Suggestion'] = data.apply(lambda row: generate_personalized_suggestion(row['Sentiment'], row['Text'], row['User_Name'], row['Purchase_History']), axis=1)

# Çıktıyı görüntüleme
print(data[['User_Name', 'Purchase_History', 'Text', 'Sentiment', 'Suggestion']].head())


  User_Name          Purchase_History  \
0     Daisy         Secret ingredient   
1   Charlie                     Taffy   
2     Alice  Vitality canned dog food   
3     Alice                Confection   
4     Daisy                     Taffy   

                                                Text Sentiment  \
0  I have bought several of the Vitality canned d...  positive   
1  Product arrived labeled as Jumbo Salted Peanut...  negative   
2  This is a confection that has been around a fe...  positive   
3  If you are looking for the secret ingredient i...  positive   
4  Great taffy at a great price.  There was a wid...  positive   

                                          Suggestion  
0  Thank you for your wonderful feedback, Daisy! ...  
1  Charlie, we're sorry to hear about your experi...  
2  Thank you for your wonderful feedback, Alice! ...  
3  Thank you for your wonderful feedback, Alice! ...  
4  Thank you for your wonderful feedback, Daisy! ...  


In [28]:
import pandas as pd
from textblob import TextBlob
import random

# Veri setini yükleme
data = pd.read_csv('/kaggle/input/amazon-product-reviews/Reviews.csv')

# Duygu analizi fonksiyonu
def analyze_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity < 0:
        return 'negative'
    else:
        return 'neutral'

# Duygu analizini uygulama
data['Sentiment'] = data['Text'].apply(analyze_sentiment)

# Kullanıcı isimleri ve alışveriş geçmişini simüle etme
user_names = ['Alice', 'Bob', 'Charlie', 'Daisy', 'Eve']
purchase_histories = ["Vitality canned dog food", "Jumbo Salted Peanuts", "Confection", "Secret ingredient", "Taffy"]

# Rastgele isim ve geçmiş ekleyerek veri çerçevesini doldurma
data['User_Name'] = [random.choice(user_names) for _ in range(len(data))]
data['Purchase_History'] = [random.choice(purchase_histories) for _ in range(len(data))]

# Geliştirilmiş öneri fonksiyonu
def generate_personalized_suggestion(sentiment, review_text, user_name, purchase_history):
    keywords = ["delivery", "quality", "taste", "packaging", "price"]
    matched_keywords = [word for word in keywords if word in review_text.lower()]

    # Negatif yorumlar için
    if sentiment == 'negative':
        if matched_keywords:
            suggestion = (f"{user_name}, we appreciate your feedback on your purchase of '{purchase_history}'. "
                          f"We're sorry to hear about issues with {', '.join(matched_keywords)}. "
                          "Our team is working to address these concerns. ")
            # Özel anahtar kelimelere göre ekstra tavsiye isteme
            if 'delivery' in matched_keywords:
                suggestion += "Could you provide more details on the delivery issue so we can improve it? "
            if 'quality' in matched_keywords:
                suggestion += "Can you tell us what aspects of the quality could be enhanced? "
            if 'taste' in matched_keywords:
                suggestion += "Could you specify what could be improved in the flavor? "
            if 'packaging' in matched_keywords:
                suggestion += "If the packaging didn't meet your expectations, let us know how we can make it better. "
            if 'price' in matched_keywords:
                suggestion += "We strive to offer value; any suggestions on pricing are welcome! "
        else:
            suggestion = (f"{user_name}, we're sorry to hear about your experience with '{purchase_history}'. "
                          "We would love to hear more details to help us improve.")
        suggestion += " Please consider reaching out to our support team for immediate assistance."

    # Nötr yorumlar için
    elif sentiment == 'neutral':
        suggestion = (f"Thank you for your insights on '{purchase_history}', {user_name}. "
                      "To enhance your experience, we recommend exploring additional features or products similar to what you've tried.")

    # Pozitif yorumlar için
    else:  # positive
        suggestion = (f"Thank you for your wonderful feedback, {user_name}! We're thrilled that you enjoyed '{purchase_history}'. "
                      "Have you tried similar products in our range? ")
        # Benzer ürün önerisi yapma
        suggestion += f"You might also enjoy our '{random.choice(purchase_histories)}'!"

    return suggestion

# Güncellenmiş öneri sütununu doldurma
data['Suggestion'] = data.apply(lambda row: generate_personalized_suggestion(row['Sentiment'], row['Text'], row['User_Name'], row['Purchase_History']), axis=1)

# Çıktıyı görüntüleme
print(data[['User_Name', 'Purchase_History', 'Text', 'Sentiment', 'Suggestion']].head())


  User_Name      Purchase_History  \
0   Charlie                 Taffy   
1   Charlie  Jumbo Salted Peanuts   
2       Eve     Secret ingredient   
3       Bob     Secret ingredient   
4   Charlie     Secret ingredient   

                                                Text Sentiment  \
0  I have bought several of the Vitality canned d...  positive   
1  Product arrived labeled as Jumbo Salted Peanut...  negative   
2  This is a confection that has been around a fe...  positive   
3  If you are looking for the secret ingredient i...  positive   
4  Great taffy at a great price.  There was a wid...  positive   

                                          Suggestion  
0  Thank you for your wonderful feedback, Charlie...  
1  Charlie, we're sorry to hear about your experi...  
2  Thank you for your wonderful feedback, Eve! We...  
3  Thank you for your wonderful feedback, Bob! We...  
4  Thank you for your wonderful feedback, Charlie...  


In [29]:
import pandas as pd
from textblob import TextBlob
import random
import pickle

# Veri setini yükleme
data = pd.read_csv('/kaggle/input/amazon-product-reviews/Reviews.csv')

# Duygu analizi fonksiyonu
def analyze_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity < 0:
        return 'negative'
    else:
        return 'neutral'

# Duygu analizini uygulama
data['Sentiment'] = data['Text'].apply(analyze_sentiment)

# Kullanıcı isimleri ve alışveriş geçmişini simüle etme
user_names = ['Alice', 'Bob', 'Charlie', 'Daisy', 'Eve']
purchase_histories = ["Vitality canned dog food", "Jumbo Salted Peanuts", "Confection", "Secret ingredient", "Taffy"]

# Rastgele isim ve geçmiş ekleyerek veri çerçevesini doldurma
data['User_Name'] = [random.choice(user_names) for _ in range(len(data))]
data['Purchase_History'] = [random.choice(purchase_histories) for _ in range(len(data))]

# Geliştirilmiş öneri fonksiyonu
def generate_personalized_suggestion(sentiment, review_text, user_name, purchase_history):
    keywords = ["delivery", "quality", "taste", "packaging", "price"]
    matched_keywords = [word for word in keywords if word in review_text.lower()]

    # Negatif yorumlar için
    if sentiment == 'negative':
        if matched_keywords:
            suggestion = (f"{user_name}, we appreciate your feedback on your purchase of '{purchase_history}'. "
                          f"We're sorry to hear about issues with {', '.join(matched_keywords)}. "
                          "Our team is working to address these concerns. ")
            if 'delivery' in matched_keywords:
                suggestion += "Could you provide more details on the delivery issue? "
            if 'quality' in matched_keywords:
                suggestion += "Can you tell us what aspects of the quality could be enhanced? "
            if 'taste' in matched_keywords:
                suggestion += "Could you specify what could be improved in the flavor? "
            if 'packaging' in matched_keywords:
                suggestion += "If the packaging didn't meet your expectations, let us know how we can make it better. "
            if 'price' in matched_keywords:
                suggestion += "We strive to offer value; any suggestions on pricing are welcome! "
        else:
            suggestion = (f"{user_name}, we're sorry to hear about your experience with '{purchase_history}'. "
                          "We would love to hear more details to help us improve.")
        suggestion += " Please consider reaching out to our support team for immediate assistance."

    # Nötr yorumlar için
    elif sentiment == 'neutral':
        suggestion = (f"Thank you for your insights on '{purchase_history}', {user_name}. "
                      "To enhance your experience, we recommend exploring additional features or products similar to what you've tried.")

    # Pozitif yorumlar için
    else:  # positive
        suggestion = (f"Thank you for your wonderful feedback, {user_name}! We're thrilled that you enjoyed '{purchase_history}'. "
                      "Have you tried similar products in our range? ")
        suggestion += f"You might also enjoy our '{random.choice(purchase_histories)}'!"

    return suggestion

# Güncellenmiş öneri sütununu doldurma
data['Suggestion'] = data.apply(lambda row: generate_personalized_suggestion(row['Sentiment'], row['Text'], row['User_Name'], row['Purchase_History']), axis=1)

# Fonksiyonları kaydetme
with open('suggestion_model.pkl', 'wb') as file:
    pickle.dump((analyze_sentiment, generate_personalized_suggestion), file)

# Çıktıyı görüntüleme
print(data[['User_Name', 'Purchase_History', 'Text', 'Sentiment', 'Suggestion']].head())


  User_Name          Purchase_History  \
0       Bob                Confection   
1       Bob  Vitality canned dog food   
2   Charlie  Vitality canned dog food   
3   Charlie                Confection   
4   Charlie                     Taffy   

                                                Text Sentiment  \
0  I have bought several of the Vitality canned d...  positive   
1  Product arrived labeled as Jumbo Salted Peanut...  negative   
2  This is a confection that has been around a fe...  positive   
3  If you are looking for the secret ingredient i...  positive   
4  Great taffy at a great price.  There was a wid...  positive   

                                          Suggestion  
0  Thank you for your wonderful feedback, Bob! We...  
1  Bob, we're sorry to hear about your experience...  
2  Thank you for your wonderful feedback, Charlie...  
3  Thank you for your wonderful feedback, Charlie...  
4  Thank you for your wonderful feedback, Charlie...  


In [30]:
# Kaydedilen fonksiyonları yükleme
with open('suggestion_model.pkl', 'rb') as file:
    analyze_sentiment, generate_personalized_suggestion = pickle.load(file)

# Test yorumu
test_text = "The quality and taste were amazing, but I expected better packaging."
sentiment = analyze_sentiment(test_text)
suggestion = generate_personalized_suggestion(sentiment, test_text, "Alice", "Taffy")

# Üretilen öneriyi yazdırma
print(suggestion)


Thank you for your wonderful feedback, Alice! We're thrilled that you enjoyed 'Taffy'. Have you tried similar products in our range? You might also enjoy our 'Vitality canned dog food'!


In [31]:
import pandas as pd
from textblob import TextBlob
import random
import pickle

# Veri setini yükleme
data = pd.read_csv('/kaggle/input/amazon-product-reviews/Reviews.csv')

# Duygu analizi fonksiyonu
def analyze_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity < 0:
        return 'negative'
    else:
        return 'neutral'

# Duygu analizini uygulama
data['Sentiment'] = data['Text'].apply(analyze_sentiment)

# Kullanıcı isimleri ve alışveriş geçmişini simüle etme
user_names = ['Alice', 'Bob', 'Charlie', 'Daisy', 'Eve']
purchase_histories = ["Vitality canned dog food", "Jumbo Salted Peanuts", "Confection", "Secret ingredient", "Taffy"]

# Rastgele isim ve geçmiş ekleyerek veri çerçevesini doldurma
data['User_Name'] = [random.choice(user_names) for _ in range(len(data))]
data['Purchase_History'] = [random.choice(purchase_histories) for _ in range(len(data))]

# Geliştirilmiş öneri fonksiyonu
def generate_personalized_suggestion(sentiment, review_text, user_name, purchase_history):
    keywords = ["delivery", "quality", "taste", "packaging", "price"]
    matched_keywords = [word for word in keywords if word in review_text.lower()]

    # Negatif yorumlar için
    if sentiment == 'negative':
        if matched_keywords:
            suggestion = (f"{user_name}, we appreciate your feedback on your purchase of '{purchase_history}'. "
                          f"We're sorry to hear about issues with {', '.join(matched_keywords)}. "
                          "Our team is working to address these concerns. ")
            # Özel anahtar kelimelere göre ekstra tavsiye isteme
            if 'delivery' in matched_keywords:
                suggestion += "Could you provide more details on the delivery issue so we can improve it? "
            if 'quality' in matched_keywords:
                suggestion += "Can you tell us what aspects of the quality could be enhanced? "
            if 'taste' in matched_keywords:
                suggestion += "Could you specify what could be improved in the flavor? "
            if 'packaging' in matched_keywords:
                suggestion += "If the packaging didn't meet your expectations, let us know how we can make it better. "
            if 'price' in matched_keywords:
                suggestion += "We strive to offer value; any suggestions on pricing are welcome! "
        else:
            suggestion = (f"{user_name}, we're sorry to hear about your experience with '{purchase_history}'. "
                          "We would love to hear more details to help us improve.")
        suggestion += " Please consider reaching out to our support team for immediate assistance."

    # Nötr yorumlar için
    elif sentiment == 'neutral':
        suggestion = (f"Thank you for your insights on '{purchase_history}', {user_name}. "
                      "To enhance your experience, we recommend exploring additional features or products similar to what you've tried.")

    # Pozitif yorumlar için
    else:  # positive
        suggestion = (f"Thank you for your wonderful feedback, {user_name}! We're thrilled that you enjoyed '{purchase_history}'. "
                      "Have you tried similar products in our range? ")
        # Benzer ürün önerisi yapma
        suggestion += f"You might also enjoy our '{random.choice(purchase_histories)}'!"

    return suggestion

# Güncellenmiş öneri sütununu doldurma
data['Suggestion'] = data.apply(lambda row: generate_personalized_suggestion(row['Sentiment'], row['Text'], row['User_Name'], row['Purchase_History']), axis=1)

# Modeli kaydetme
model = None  # Burada modelinizi tanımlamanız gerekiyor (örneğin bir sınıflandırma modeli)
with open('naive_bayes_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

# TF-IDF vektörleştiricisini kaydetme (örneğin kullanıyorsanız)
tfidf_vectorizer = None  # Burada TF-IDF vektörleştiricisini tanımlayın
with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(tfidf_vectorizer, vectorizer_file)

# Verileri CSV olarak kaydetme
data.to_csv('personalized_suggestions.csv', index=False)

# Dosyaları Kaggle ortamından indirmenizi sağlayın
import shutil

# Dosyaları yerel dizine taşımak için
shutil.move('naive_bayes_model.pkl', '/kaggle/working/naive_bayes_model.pkl')
shutil.move('tfidf_vectorizer.pkl', '/kaggle/working/tfidf_vectorizer.pkl')
shutil.move('personalized_suggestions.csv', '/kaggle/working/personalized_suggestions.csv')


'/kaggle/working/personalized_suggestions.csv'

In [32]:
import os

# Çalışma dizinindeki dosyaları listeleme
print(os.listdir('/kaggle/working'))


['__notebook__.ipynb', 'suggestion_model.pkl', 'tfidf_vectorizer.pkl', 'personalized_suggestions.csv', 'naive_bayes_model.pkl']


In [33]:
import joblib
joblib.dump(nb_model, '/kaggle/working/nb_model.pkl')


['/kaggle/working/nb_model.pkl']

In [34]:
import os
print(os.path.exists('/kaggle/working/nb_model.pkl'))


True


In [35]:
import os

# /kaggle/working dizinindeki dosyaları listele
print(os.listdir('/kaggle/working'))


['__notebook__.ipynb', 'suggestion_model.pkl', 'tfidf_vectorizer.pkl', 'personalized_suggestions.csv', 'nb_model.pkl', 'naive_bayes_model.pkl']


In [36]:
import zipfile

# ZIP dosyasını /kaggle/working dizininde oluşturuyoruz
with zipfile.ZipFile('/kaggle/working/nb_model.zip', 'w') as zipf:
    zipf.write('/kaggle/working/nb_model.pkl', arcname='nb_model.pkl')


In [37]:
import joblib

# Modelinizi kaydedin
joblib.dump(nb_model, '/kaggle/working/nb_model.pkl')
print("Model başarıyla kaydedildi.")


Model başarıyla kaydedildi.


In [38]:
import os

# /kaggle/working dizinindeki dosyaları listeleyin
print(os.listdir('/kaggle/working'))


['__notebook__.ipynb', 'suggestion_model.pkl', 'tfidf_vectorizer.pkl', 'personalized_suggestions.csv', 'nb_model.pkl', 'naive_bayes_model.pkl', 'nb_model.zip']


In [39]:
import os
print(os.listdir('/kaggle/working'))


['__notebook__.ipynb', 'suggestion_model.pkl', 'tfidf_vectorizer.pkl', 'personalized_suggestions.csv', 'nb_model.pkl', 'naive_bayes_model.pkl', 'nb_model.zip']


In [40]:
from IPython.display import FileLink

# Dosyanın tam yolu
FileLink('/kaggle/working/nb_model.zip')


/kaggle/working/nb_model.zip

In [41]:
import joblib

# Vektörizer'i kaydetme
joblib.dump(tfidf_vectorizer, '/kaggle/working/tfidf_vectorizer.pkl')
print("Vektörizer başarıyla kaydedildi.")


Vektörizer başarıyla kaydedildi.


In [42]:
import os

# /kaggle/working dizinini listele
print(os.listdir('/kaggle/working'))


['__notebook__.ipynb', 'suggestion_model.pkl', 'tfidf_vectorizer.pkl', 'personalized_suggestions.csv', 'nb_model.pkl', 'naive_bayes_model.pkl', 'nb_model.zip']


In [43]:
# Vektörizer'i yükleme
loaded_vectorizer = joblib.load('/kaggle/working/tfidf_vectorizer.pkl')

# Yüklenen vektörizer'in tipini kontrol et
print("Vektörizer tipi:", type(loaded_vectorizer))


Vektörizer tipi: <class 'NoneType'>


.")


In [44]:
print(type(tfidf_vectorizer))


<class 'NoneType'>


In [45]:
print(data.columns)


Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text',
       'Sentiment', 'User_Name', 'Purchase_History', 'Suggestion'],
      dtype='object')


In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

# Vektörizeri oluşturun ve eğitin
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Gereken parametreleri ayarlayın
tfidf_vectorizer.fit(data['Text'])  # 'Text' sütununu kullanıyoruz

print("Vektörizer başarıyla eğitildi.")

# Vektörizeri kaydedin
joblib.dump(tfidf_vectorizer, '/kaggle/working/tfidf_vectorizer.pkl')
print("Vektörizer başarıyla kaydedildi.")


Vektörizer başarıyla eğitildi.
Vektörizer başarıyla kaydedildi.


In [47]:
import os

# /kaggle/working dizinindeki dosyaları listeleyin
print("Dizin içeriği:", os.listdir('/kaggle/working'))


Dizin içeriği: ['__notebook__.ipynb', 'suggestion_model.pkl', 'tfidf_vectorizer.pkl', 'personalized_suggestions.csv', 'nb_model.pkl', 'naive_bayes_model.pkl', 'nb_model.zip']
